In [9]:
f = open("temp_text_puh.txt", "r")
template_str = read(f, String)
close(f)

input_str = template_str[1:500]
input_str = '<' * input_str * '>'
println(length(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
println(letters)

502
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']


In [10]:
# generate SDR

SDR_len = 128 # 128
sparsity = 4 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / SDR_len
println("sparse_prob = $sparse_prob")

letter_SDRs = Dict{Char, Array}()

for letter in letters
    rnd_SDR = zeros(SDR_len)

    while true
        rnd_SDR = rand(SDR_len) .< sparse_prob

        if sum(rnd_SDR .> 0) != sparsity
            continue
        end
        
        is_over_max = false
        for SDR in values(letter_SDRs)
            if sum(SDR .& rnd_SDR) > max_overlap
                is_over_max = true
                break
            end
        end
        
        if is_over_max
            continue
        end
            
        break
    end

    # println(letter)
    letter_SDRs[letter] = rnd_SDR
end

# for (letter, SDR) in letter_SDRs
#     sum_SDR = sum(SDR)
#     println("$letter : $sum_SDR $SDR")
# end

sparse_prob = 0.03125


In [11]:
# text to SDRs
input_SDRs = []

for inp in input_str
    push!(input_SDRs, letter_SDRs[inp])
end

len_inp_SDRs = length(input_SDRs)
println("len_inp_SDRs=$len_inp_SDRs")

len_inp_SDRs=502


In [12]:
typeof(rand(0:32))

Int64

In [13]:
# init start column
column_size = 32

start_SDR = letter_SDRs['<']

start_I = zeros(length(start_SDR), column_size)
println(typeof(start_I))

active_start_SDR_idx = findall(start_SDR .== 1)
# println(typeof(active_start_SDR_idx))

for as_idx in active_start_SDR_idx
    rnd_active = rand(1:column_size)
    # println(typeof(as_idx), typeof(rnd_active))

    start_I[as_idx, rnd_active] = 1
end

println(length(start_I[128,:]))
println(sum(start_I .> 0))

# for col in eachcol(start_I)
#     println(Int64.(col))
# end

Matrix{Float64}
32
4


In [14]:
function train_one(Inp, W, V, SDR_len,column_size,prec,SDR, eps0, eps0_V)
    rsh_I = reshape(Inp, (1,1,SDR_len * column_size) )

    O = sum(W .* rsh_I, dims=3)
    # O = sum((@~ W .* rsh_I), dims = 3)
    # println(size(O, 1),' ', size(O,2),' ', size(O,3))

    O ./= maximum(abs.(O)) + prec
    # println(size(O, 1),' ', size(O,2),' ', size(O,3))
    
    nV = abs.(V .- reshape(maximum(V, dims=2), (1, SDR_len))')
    nV ./= reshape(maximum(nV, dims=2), (1, SDR_len))' .+ prec

    # println(size(nV, 1),' ', size(nV,2),' ', size(nV,3))

    OV = O .+ (nV .* eps0_V)

    OV .*= reshape(SDR, (1, size(SDR, 1)))'
    max_O = maximum(OV, dims=2)
    # println(size(max_O))
    # println(max_O)

    O .*= reshape(SDR, (1, size(SDR, 1)))'
    # println(size(O))
    # println(sum(O .> 0))
    # O[OV < max_O[np.newaxis].T] = 0
    max_O_T = reshape(max_O, (1, size(max_O, 1)))'
    # println(size(max_O_T))
    # println(sum(OV .< max_O_T))
    O[OV .< max_O_T] .= 0
    # println(sum(O .> 0))
    # O .*= (OV .>= reshape(max_O, (1, size(max_O, 1)))')
    O = reshape(O, (size(O,1), size(O,2)))
    max_O_idx = findall(O .!= 0)
    # println(max_O_idx)
    
    mul = W[max_O_idx,:] .* O[max_O_idx]
    # println("mul ", size(mul))
    # println("rsh_I ", size(rsh_I))
    rsh_I = reshape(rsh_I, (1, SDR_len * column_size))
    minu = rsh_I .- mul
    # println("minu ", size(minu))
    # O = reshape(O, (size(O,1), size(O,2), 1))
    W[max_O_idx,:] .+= (rsh_I .- W[max_O_idx,:] .* O[max_O_idx]) .* eps0

    # winners
    V[max_O_idx] .+= 1

    # recur
    O[O .!= 0] .= 1

    Inp = copy(O)

    return Inp
end

train_one (generic function with 1 method)

In [15]:
# import Pkg;
# Pkg.add("LazyArrays")

using LazyArrays, LinearAlgebra

Inp = zeros(SDR_len, column_size)
O = zeros(SDR_len, column_size)

W = rand(Float64, (SDR_len, column_size, SDR_len * column_size)) * 1e-2

V = zeros(SDR_len, column_size)

prec = 1e-30

# train

epoch_cnt = 1

eps0 = 0.003

eps0_V = 0.09

for epoch_idx in range(1, epoch_cnt)
    Inp = copy(start_I)

    for SDR in input_SDRs[1:end]
        Inp = train_one(Inp, W, V, SDR_len, column_size, prec, SDR, eps0, eps0_V)
    end
end
# O.shape
# println(mean(V, axis=1))
println(sum(V .== 0))
println(sum((V .> 0), dims=2))
# for col in np.rot90(V):
#     print(f'{col.astype(int).tolist()}')

2959
[0; 7; 0; 0; 11; 13; 0; 2; 3; 25; 24; 32; 0; 0; 20; 0; 1; 0; 0; 24; 16; 0; 0; 0; 0; 0; 0; 32; 0; 0; 7; 26; 0; 9; 0; 30; 24; 0; 0; 0; 8; 20; 23; 32; 0; 0; 0; 9; 0; 16; 0; 17; 0; 0; 1; 13; 25; 19; 20; 0; 1; 0; 1; 29; 0; 0; 0; 0; 31; 32; 0; 8; 21; 0; 13; 3; 0; 0; 19; 0; 0; 0; 10; 8; 22; 13; 13; 0; 1; 0; 23; 9; 7; 2; 11; 8; 28; 8; 21; 0; 7; 32; 0; 0; 0; 0; 18; 23; 21; 0; 0; 0; 1; 29; 0; 0; 0; 17; 11; 11; 0; 25; 25; 19; 0; 23; 0; 24;;]


In [16]:
# test
error = []

I = np.copy(start_I)

for SDR in input_SDRs[1:]:
    rsh_I = I.reshape(SDR_len * column_size)

    O = np.dot(W, rsh_I)

    max_O = np.max(O, axis=1)
    argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

    SDR_O = np.zeros(O.shape[0])
    SDR_O[argmax_O_idx] = 1

    err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
    # print(f'{err=}')
    error.append(err)

    O *= SDR[np.newaxis].T
    max_O = np.max(O, axis=1)
    O[O < max_O[np.newaxis].T] = 0

    # recur
    O[O != 0] = 1

    I = np.copy(O)

error = np.array(error)
mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')

LoadError: UndefVarError: `np` not defined